In [ ]:
import torch
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, RGCNConv

from utils.path_manage import get_files

In [ ]:
triplets, lookup, ASD_dictionary, BCE_dictionary, Edge_list, Edge_features = get_files()
edge_index = torch.tensor(Edge_list, dtype=torch.long)
edge_features = torch.tensor(Edge_features, dtype=torch.float)
edge_type = torch.tensor([x.index(max(x)) for x in Edge_features], dtype=torch.long)

In [ ]:
print(edge_index[0:10])
print(triplets[0])

In [ ]:
data = Data(edge_index=edge_index[0:10].t().contiguous(), edge_attr=edge_features[0:10], edge_type=edge_type[0:10])

In [ ]:
data.edge_index[1]

In [ ]:
# this code is designed for graph classification not link prediction. 

class Net(torch.nn.Module):
    def __init__(self, data, hidden_dim):
        super(Net, self).__init__()

# Should add embedding layer at start

        self.conv1 = RGCNConv(in_channels=data.num_nodes, out_channels=hidden_dim, num_relations=data.num_edge_features, num_bases=30)
        self.conv2 = RGCNConv(in_channels=hidden_dim, out_channels=hidden_dim, num_relations=data.num_edge_features, num_bases=30)
        
        
        # should maybe make this an embedding layer too? idk. 
        self.w_relation = torch.nn.Parameter(torch.Tensor(data.num_edge_features+1, hidden_dim))

    def forward(self, edge_index, edge_type):

        # Should ad embedding layer 
        x = F.relu(self.conv1(None, data.edge_index, data.edge_type))
        x = self.conv2(x, edge_index, edge_type)
        return x

    def distmult(self, embedding, triplets):
        s = embedding[triplets[:,0]]
        print(s)
        r = self.w_relation[triplets[:,1]]
        print(r)
        o = embedding[triplets[:,2]]
        print(o)
        score = torch.sum(s * r * o, dim=1)
        return score
        


model = Net(data=data, hidden_dim = 100)
model.parameters

In [ ]:
out = model(data.edge_index, data.edge_type)

In [ ]:
triplets[0:2]

In [ ]:
distmult = model.distmult(out, triplets[0:1])
print(distmult)

In [ ]:
model

In [ ]:
for param in model.parameters():
    print(type(param.data), param.size())

In [ ]:
out = model(data.edge_index, data.edge_type)
# this is a call to the forward function!!!

In [ ]:
# okay so your model gives out node embeddings. Now to do a distmult on these bad boys.  

In [ ]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Entities
from torch_geometric.nn import RGCNConv

name = 'MUTAG'
path = osp.join(
    osp.dirname(osp.realpath('__file__')), '..', 'data', 'Entities', name)
dataset = Entities(path, name)
data = dataset[0]

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = RGCNConv(
            data.num_nodes, 16, dataset.num_relations, num_bases=30)
        self.conv2 = RGCNConv(
            16, dataset.num_classes, dataset.num_relations, num_bases=30)

    def forward(self, edge_index, edge_type, edge_norm):
        x = F.relu(self.conv1(None, edge_index, edge_type))
        x = self.conv2(x, edge_index, edge_type)
        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
print(model.parameters)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0005)


def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.edge_index, data.edge_type, data.edge_norm)
    print(out)
    F.nll_loss(out[data.train_idx], data.train_y).backward()
    print(data.train_idx)
    print(out[data.train_idx])
    optimizer.step()